In [24]:
# 📘 Introducción a Apache Spark SQL (Páginas 1 a 11)
# Este notebook es una guía práctica basada en los contenidos teóricos del PDF sobre Apache Spark,
# desde la introducción hasta los motores de optimización Catalyst y Tungsten.


# ✅ Paso 1: Instalación de PySpark en Colab (solo si no está instalado)
!pip install -q pyspark

In [26]:
# ✅ Paso 2: Crear la sesión en Apache Spark (SparkSession)
from pyspark.sql import SparkSession

# Crear la sesión
spark = SparkSession.builder.appName("ProyectoMetroSantiago").config("spark.executor.memory", "1g").config("spark.driver.memory", "1g").config("spark.sql.shuffle.partitions","2").getOrCreate()

In [27]:
# ✅ Paso 3: Cargar datos estructurados como un DataFrame
# Asegúrate de subir el archivo 'metro_santiago.csv' a tu entorno de Google Colab

ruta_csv = "metro_santiago.csv"
df = spark.read.csv(ruta_csv, header=True, inferSchema=True)

In [28]:
# ✅ Paso 4: Exploración inicial del DataFrame
df.show()

+---------+-----+--------------+--------------------+----------------+-------------------+
|id_evento|linea|      estacion|      tipo_incidente|duracion_minutos|               hora|
+---------+-----+--------------+--------------------+----------------+-------------------+
|        1|   L2|      Lo Prado|     Falla eléctrica|              15|2025-08-25 11:45:00|
|        2|   L5|   Irarrázaval|          Evacuación|              15|2025-08-25 20:00:00|
|        3|   L4|     San Pablo|     Falla eléctrica|              25|2025-08-25 19:00:00|
|        4|   L1|      Lo Prado|     Falla eléctrica|              30|2025-08-25 05:45:00|
|        5|   L3|   Irarrázaval|     Falla eléctrica|              20|2025-08-25 08:00:00|
|        6|   L2|    Los Leones|       Mantenimiento|              30|2025-08-25 17:00:00|
|        7|   L3|         Ñuñoa|       Mantenimiento|               5|2025-08-25 12:30:00|
|        8|   L1|    Los Héroes|     Falla eléctrica|              30|2025-08-25 08:30:00|

In [29]:
df.printSchema()

root
 |-- id_evento: integer (nullable = true)
 |-- linea: string (nullable = true)
 |-- estacion: string (nullable = true)
 |-- tipo_incidente: string (nullable = true)
 |-- duracion_minutos: integer (nullable = true)
 |-- hora: timestamp (nullable = true)



In [30]:
print("Columnas del DataFrame:", df.columns)
df.describe().show()


Columnas del DataFrame: ['id_evento', 'linea', 'estacion', 'tipo_incidente', 'duracion_minutos', 'hora']
+-------+------------------+-----+---------+--------------+-----------------+
|summary|         id_evento|linea| estacion|tipo_incidente| duracion_minutos|
+-------+------------------+-----+---------+--------------+-----------------+
|  count|                50|   50|       50|            50|               50|
|   mean|              25.5| NULL|     NULL|          NULL|             17.8|
| stddev|14.577379737113251| NULL|     NULL|          NULL|7.964205637009345|
|    min|                 1|   L1|Baquedano|    Evacuación|                5|
|    max|                50|   L6|    Ñuñoa|       Retraso|               30|
+-------+------------------+-----+---------+--------------+-----------------+



In [31]:
print("El DataFrame contiene", df.count(), "registros.")
print("Primera fila:", df.first())
df.select("tipo_incidente").distinct().show()

El DataFrame contiene 50 registros.
Primera fila: Row(id_evento=1, linea='L2', estacion='Lo Prado', tipo_incidente='Falla eléctrica', duracion_minutos=15, hora=datetime.datetime(2025, 8, 25, 11, 45))
+--------------------+
|      tipo_incidente|
+--------------------+
|     Falla eléctrica|
|          Evacuación|
|       Mantenimiento|
|Interrupción de s...|
|             Retraso|
+--------------------+



In [32]:
# ✅ Paso 5: Uso de DataFrames para consultas
# Podemos usar métodos como select, filter, groupBy, orderBy para consultar los datos


# Ejemplo 1: Mostrar las estaciones donde ocurrió un tipo específico de incidente
df.filter(df.tipo_incidente == "Falla eléctrica").select("estacion", "tipo_incidente").show(5)

+-----------+---------------+
|   estacion| tipo_incidente|
+-----------+---------------+
|   Lo Prado|Falla eléctrica|
|  San Pablo|Falla eléctrica|
|   Lo Prado|Falla eléctrica|
|Irarrázaval|Falla eléctrica|
| Los Héroes|Falla eléctrica|
+-----------+---------------+
only showing top 5 rows



In [33]:
# Ejemplo 2: Contar cuántos incidentes ocurrieron por tipo
df.groupBy("tipo_incidente").count().orderBy("count", ascending=False).show()

+--------------------+-----+
|      tipo_incidente|count|
+--------------------+-----+
|     Falla eléctrica|   14|
|       Mantenimiento|   12|
|Interrupción de s...|   11|
|          Evacuación|    8|
|             Retraso|    5|
+--------------------+-----+



In [34]:
# ✅ Paso 6: Operaciones avanzadas con DataFrames
# Podemos usar funciones de agregación, filtrado compuesto y ordenamientos múltiples
from pyspark.sql.functions import col, desc, count

# Ejemplo: Agrupar por estación y contar incidentes mayores a una cantidad
df.groupBy("estacion").agg(count("tipo_incidente").alias("total_incidentes")) \
.filter(col("total_incidentes") > 3) \
.orderBy(desc("total_incidentes")) \
.show()

+-----------+----------------+
|   estacion|total_incidentes|
+-----------+----------------+
| Los Héroes|               6|
|  Baquedano|               6|
|Irarrázaval|               4|
+-----------+----------------+



In [36]:
# ✅ Paso 7: Optimización de consultas - Catalyst
# Catalyst es el motor de optimización lógica y física de Spark SQL.
# Aunque no lo vemos directamente, Catalyst reordena y reescribe nuestras consultas
# para ejecutarlas de forma más eficiente.


# Creamos una vista temporal para permitir consultas SQL
# Esto facilita la comprensión para quienes vienen del mundo relacional
df.createOrReplaceTempView("incidentes")


# Consulta con SQL optimizada por Catalyst
spark.sql("""
  SELECT estacion, COUNT(*) as total
  FROM incidentes
  WHERE tipo_incidente = 'Falla eléctrica'
  GROUP BY estacion
  ORDER BY total DESC
  LIMIT 5
""").show()

+--------------+-----+
|      estacion|total|
+--------------+-----+
|      Lo Prado|    2|
|    Los Héroes|    2|
|Vicente Valdés|    2|
|     San Pablo|    2|
|     Cerrillos|    1|
+--------------+-----+



In [37]:
# ✅ Paso 8: Optimización de consultas - Tungsten
# Tungsten es un motor de ejecución física que optimiza el uso de memoria y CPU
# No requiere ningún código especial, actúa automáticamente en segundo plano
# mientras ejecutamos operaciones sobre DataFrames o consultas SQL.


# Ejemplo adicional con Tungsten optimizando internamente
spark.sql("""
SELECT linea, COUNT(*) as cantidad
FROM incidentes
GROUP BY linea
ORDER BY cantidad DESC
""").show()

+-----+--------+
|linea|cantidad|
+-----+--------+
|   L5|      13|
|   L3|       9|
|  L4A|       9|
|   L1|       8|
|   L2|       5|
|   L4|       5|
|   L6|       1|
+-----+--------+



In [38]:
# ✅ Paso 9: Comparación entre DataFrame y Dataset
# En PySpark, no existe directamente Dataset como en Scala. Todo se trabaja con DataFrames.
# Podemos pensar en el DataFrame de PySpark como una abstracción similar al Dataset de Scala + Java.


# Mostramos el esquema y los primeros registros
df.printSchema()
df.show(5)

root
 |-- id_evento: integer (nullable = true)
 |-- linea: string (nullable = true)
 |-- estacion: string (nullable = true)
 |-- tipo_incidente: string (nullable = true)
 |-- duracion_minutos: integer (nullable = true)
 |-- hora: timestamp (nullable = true)

+---------+-----+-----------+---------------+----------------+-------------------+
|id_evento|linea|   estacion| tipo_incidente|duracion_minutos|               hora|
+---------+-----+-----------+---------------+----------------+-------------------+
|        1|   L2|   Lo Prado|Falla eléctrica|              15|2025-08-25 11:45:00|
|        2|   L5|Irarrázaval|     Evacuación|              15|2025-08-25 20:00:00|
|        3|   L4|  San Pablo|Falla eléctrica|              25|2025-08-25 19:00:00|
|        4|   L1|   Lo Prado|Falla eléctrica|              30|2025-08-25 05:45:00|
|        5|   L3|Irarrázaval|Falla eléctrica|              20|2025-08-25 08:00:00|
+---------+-----+-----------+---------------+----------------+---------------

In [39]:
# ✅ Paso 10: Interoperabilidad entre DataFrame y Dataset
# En PySpark, no se trabaja con tipos estrictos como en Scala (case class),
# pero sí podemos convertir entre estructuras con RDD y DataFrame


# Convertir un DataFrame a un RDD
df_rdd = df.rdd
print(df_rdd.take(3))

[Row(id_evento=1, linea='L2', estacion='Lo Prado', tipo_incidente='Falla eléctrica', duracion_minutos=15, hora=datetime.datetime(2025, 8, 25, 11, 45)), Row(id_evento=2, linea='L5', estacion='Irarrázaval', tipo_incidente='Evacuación', duracion_minutos=15, hora=datetime.datetime(2025, 8, 25, 20, 0)), Row(id_evento=3, linea='L4', estacion='San Pablo', tipo_incidente='Falla eléctrica', duracion_minutos=25, hora=datetime.datetime(2025, 8, 25, 19, 0))]


In [40]:
# Convertir un RDD a un DataFrame (requiere esquema)
from pyspark.sql import Row
rdd_to_df = df_rdd.map(lambda x: Row(estacion=x['estacion'], tipo_incidente=x['tipo_incidente']))
new_df = spark.createDataFrame(rdd_to_df)
new_df.show(5)

+-----------+---------------+
|   estacion| tipo_incidente|
+-----------+---------------+
|   Lo Prado|Falla eléctrica|
|Irarrázaval|     Evacuación|
|  San Pablo|Falla eléctrica|
|   Lo Prado|Falla eléctrica|
|Irarrázaval|Falla eléctrica|
+-----------+---------------+
only showing top 5 rows



In [42]:
# ✅ Paso 11: Funciones definidas por el usuario (UDFs)
# Las UDFs permiten definir funciones propias usando Python y aplicarlas sobre columnas del DataFrame


from pyspark.sql.functions import udf
from pyspark.sql.types import StringType


# UDF para clasificar incidentes como técnicos o no técnicos
def clasificar_tipo(tipo):
  if "técnica" in tipo.lower():
    return "Técnico"
  else:
    return "Otro"


# Registrar la UDF en Spark
clasificar_udf = udf(clasificar_tipo, StringType())

In [43]:
# Aplicar la UDF sobre el DataFrame
df_udf = df.withColumn("categoria", clasificar_udf(df["tipo_incidente"]))
df_udf.select("tipo_incidente", "categoria").show(5)

+---------------+---------+
| tipo_incidente|categoria|
+---------------+---------+
|Falla eléctrica|     Otro|
|     Evacuación|     Otro|
|Falla eléctrica|     Otro|
|Falla eléctrica|     Otro|
|Falla eléctrica|     Otro|
+---------------+---------+
only showing top 5 rows



In [45]:
# ✅ Paso 12: Registro de UDFs como funciones SQL
# Ahora registramos la UDF anterior para usarla en sentencias SQL


spark.udf.register("clasificarSQL", clasificar_tipo, StringType())
df_udf.createOrReplaceTempView("incidentes_udf")

# Consulta usando la UDF personalizada desde SQL
spark.sql("""
SELECT tipo_incidente, clasificarSQL(tipo_incidente) AS categoria
FROM incidentes_udf
LIMIT 5
""").show()

+---------------+---------+
| tipo_incidente|categoria|
+---------------+---------+
|Falla eléctrica|     Otro|
|     Evacuación|     Otro|
|Falla eléctrica|     Otro|
|Falla eléctrica|     Otro|
|Falla eléctrica|     Otro|
+---------------+---------+



In [47]:
# ✅ Paso 13: Interoperación entre RDDs y DataFrames
# Transformación desde RDD con map hacia un DataFrame estructurado


rdd_simplificado = df_rdd.map(lambda x: (x['estacion'], x['tipo_incidente']))


schema = ["estacion", "tipo_incidente"]
from pyspark.sql import SparkSession
inter_df = spark.createDataFrame(rdd_simplificado, schema)


inter_df.show(5)

+-----------+---------------+
|   estacion| tipo_incidente|
+-----------+---------------+
|   Lo Prado|Falla eléctrica|
|Irarrázaval|     Evacuación|
|  San Pablo|Falla eléctrica|
|   Lo Prado|Falla eléctrica|
|Irarrázaval|Falla eléctrica|
+-----------+---------------+
only showing top 5 rows

